In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [2]:
#LeNet-inspired 1D CNN
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error
# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']


# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LeNet model
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=5, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions on new data
predictions = model.predict(X_test)
ypred=predictions
print('Predictions:', predictions)
ypred = ypred.flatten()


Epoch 1/50
1131/1131 [==============================] - 4s 3ms/step - loss: 0.0082
Epoch 2/50
1131/1131 [==============================] - 3s 3ms/step - loss: 0.0014
Epoch 3/50
1131/1131 [==============================] - 3s 2ms/step - loss: 0.0012
Epoch 4/50
1131/1131 [==============================] - 3s 3ms/step - loss: 0.0011
Epoch 5/50
1131/1131 [==============================] - 3s 3ms/step - loss: 9.5982e-04
Epoch 6/50
1131/1131 [==============================] - 3s 3ms/step - loss: 9.0900e-04
Epoch 7/50
1131/1131 [==============================] - 3s 3ms/step - loss: 8.2670e-04
Epoch 8/50
1131/1131 [==============================] - 3s 3ms/step - loss: 8.3147e-04
Epoch 9/50
1131/1131 [==============================] - 3s 3ms/step - loss: 7.6461e-04
Epoch 10/50
1131/1131 [==============================] - 3s 3ms/step - loss: 7.4757e-04
Epoch 11/50
1131/1131 [==============================] - 3s 3ms/step - loss: 7.3520e-04
Epoch 12/50
1131/1131 [==============================] - 

In [3]:
#For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

MAE = 0.02 (%)
MAPE = 1.30 (%)
RMSE = 0.02 (%)
